<a href="https://colab.research.google.com/github/GizemGno/WordEmbedding_ColabNotebook/blob/master/WordEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook'un orijinali: https://www.tensorflow.org/tutorials/text/word_embeddings

Makine Öğrenmesi ile metin işlerken girdiyi modele sayı vektörleri şeklinde veriyoruz. Bunun için birkaç yol var, referans aldığımız tensorflow notebook'u üç stratejiyi ele almış.


**One-Hot Encoding**

Bu yöntemde her bir benzersiz kelimeyi one-hot vektöler ile ifade ediyoruz. Burada vektör boyutu sözlük boyutu kadar ve her birinde ayrı bir indeks dışındaki tüm indeksler sıfır oluyor. 

Mesela örnek cümlemiz "İki kere iki dört eder" olsun. Burada sözlüğümüzdeki kelimeler (iki, kere, dört, eder), vektör boyutumuz 4, örnek gösterim:

iki = [1 0 0 0]

kere = [0 1 0 0]

dört = [0 0 1 0]

eder = [0 0 0 1]



Cümleyi kodlamak için içerdiği kelimelerin one-hot vektörlerini birleştirmemiz yeterli olacak. 

Vektörlerdeki elementlerin çoğu 0 oluduğu için (bu vektörlere sparse vektör deniyor) bu yöntem verimsiz bir yöntem. 

**Her Kelimeyi Bir Sayı ile Gösterme**

Bu yöntemde one-hot encode vektörler yerine her kelimeyi bir sayı ile gösteriyoruz. Mesela önceki örnekteki cümleyi (1,2,1,3,4) şeklinde ifade edebiliriz. Artık sparse değil dense vektör elde ettiğimizden yöntem daha verimli. 

Bu yöntemin dezavantaji ise kelimelerin arasındaki ilişkiyi yakalayamıyor olması. 


**Word Embeddings**

Word embeddings ile ifade edilen kelimeler hem dense oluyor, hem de kelimelerin arasındaki ilişki yakalanmış oluyor. Burada vektör uzunluğu belirlediğimiz parametre sayısı kadar ve her indis yüzdelik bir sayı oluyor. Yöntemin en iyi yanlarından biri bu değerleri el ile vermememiz, bunun yerine bu değerler modelin eğitilmesi sırasında öğreniliyor. Küçük veri setlerinde 8 boyutlu embeddingler yaygın görülürken, daha büyük veri setlerinde bu sayı 1024'e kadar çıkabiliyor. 

*4 Boyutlu Embedding Örneği*

iki => [1.2 -0.1 4.3 3.2]
kere => [0.4 2.5 -0.9 0.5]
dört => [2.1 0.3 0.1 0.4]
...

In [0]:
 # Setup
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # Tensorflow 2.0 versiyonunu kullanmak için
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

#tensorflow dataset
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

**Keras Embedding Katmanı**

Embedding katmanı belirli bir kelimeye karşılık gelen tamsayıları dense vektörlere eşleyen bir lookup tablosu olarak düşünülebilir. Embedding'in boyutu bir hyperparameter, uygulanacak probleme en uygun değer test edilerek bulunmalı. 

Embedding katman oluşturulduğunda, katmanın ağırlıklarına rasgale değerler atanıyor. Modelin eğitilmesi sırasında backpropagation ile bu ağırlıklar güncelleniyor. Eğitildikten sonra, öğrenilen kelime vektörleri kelimeler arasındaki benzerlikleri kabaca kodlamış oluyor. 

In [3]:
# Tensorflow Dataset'ten imdb yorumlarini yukleyelim

(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/0.1.0. Subsequent calls will reuse this data.


In [0]:
# Film yorumları herhangi uzunlukta olabilir. 
# padded_batch metodunu kullanarak uzunlukları standart hale getirecegiz

padded_shapes = ([None],())
train_batches = train_data.shuffle(1000).padded_batch(10, padded_shapes = padded_shapes)
test_batches = test_data.shuffle(1000).padded_batch(10, padded_shapes = padded_shapes)

In [8]:
embedding_dim=16
encoder = info.features['text'].encoder

model = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  layers.GlobalAveragePooling1D(),
  layers.Dense(1, activation='sigmoid')
])

model.summary()

# Modeli compile edip eğitelim

model.compile(optimizer='adam',
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_batches,
    epochs = 10,
    validation_data = test_batches, validation_steps=20)

# Burada accuracy ve loss kısımlarına dikkat ederek modelin performansına 
# bakabiliriz

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 130,977
Trainable params: 130,977
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2500/2500 [==============================] - 23s 9ms/step - loss: 0.6360 - accuracy: 0.6926 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
2500/2500 [==============================] - 16s 6ms/step - loss: 0.4635 - accuracy: 0.8361 - val_loss: 0.3938 - val_accuracy: 0.8700
Epoch 3/10
2500/2500 [==============================] - 16s 6ms/step - loss: 0.3

In [0]:
# Ogrenilen word embeddingler (vocab_size, embedding_dim) boyutunda bir matris 
# Asagidaki kod ile bu embeddingleri bir degiskene atabiliriz

e = model.layers[0]
weights = e.get_weights()[0]

# Asagidaki kod ile agirliklari diske yazabiliriz

import io


out_v = io.open('vecs.tsv','w',encoding='utf-8')
out_m = io.open('meta.tsv','w',encoding='utf-8')

for num, word in enumerate(encoder.subwords):
  vec = weights[num+1] # 0 numara padding oldugundan atliyoruz
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

In [0]:
# Colab'da calisiyorsaniz asagidaki kod ile az once olusturdugumuz
# dosyalari local bilgisayariniza indirip tensorflow projector ile 
# modelinizi gorsel olarak inceleyebilirsiniz

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')